# TOTEMS - Tidal Orbital decay Timing Extrapolation & Modelling Software



In [8]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit #Import curve fitting (Which is what we need for tidal decay)

import pylightcurve as plc # import pylightcurve - used for BJD HJD conversions - Angelos Tsiaras

In [10]:
%matplotlib notebook
# if this isn't in a seperate cell, sometimes doesn't work right
# magic commands are weird, to say the least.




First function: gets the relevant data for the exoplanet from the Exoplanet Transit Database. Use this if you don't have any archive data of your own to use, though really any data from a recent paper will probably be better.

In [11]:
def get_etd_data(ra, dec, url):
    """Imports the archive data from ETD. Uses the RA & Dec of a target to convert HJD_UTC to BJD_TDB.
    Inputs:
    - ra, the RA in hh mm ss.ss
    - dec, the Declination in hh mm ss.ss
    - url, the URL of datafile
    Outputs:
    - no, the number of the entry in ETD's list
    - bjd, t_mid converted to BJD_TDB
    - mid_err, the uncertainty in t_mid
    - epoch, the epoch number for the transit based on the t_0 and period on ETD.
    - dq, ETD's Data Quality measurement. 1 is best, 5 is worst.
    """
    # TODO import the epoch and period guess from ETD automatically, perhaps using split?

    # Import the data into a series of arrays using numpy loadtxt. Note we skip the first 5 rows and only use
    # certain columns, because there's lots of extra stuff we aren't using
    no,hjd,mid_err,epoch,dq = np.genfromtxt(url, encoding='unicode_escape', delimiter=';', skip_header=5, missing_values='', filling_values=-1, usecols=(0,1,2,3,10), unpack=True)

    # convert from the truncated HJD to the full HJD
    hjd = hjd + 2400000

    # assuming ETD uses HJD_UTC, convert to BJD_TDB.
    # of course, it might be worth checking that each individual data point specified is ACTUALLY HJD,
    # as I don't think ETD vets for those kinds of mistakes

    ra_dec_string = ra+" "+dec #concatenate into one string for the hh mm ss.ss to deg convert function
    ra, dec = plc.ra_dec_string_to_deg(ra_dec_string) # convert from hh mm ss.ss to degrees

    # Convert to BJD_TDB. N.B. we assume uncertainty remains the same.
    bjd = np.zeros(len(hjd)) # make a zeros array to store BJD in
        for i in tqdm(range(0, len(hjd))):
        bjd[i] = plc.hjd_utc_to_bjd_tdb(ra, dec, hjd[i]) # do the actual converting

    # return the data arrays. Notice this means you end up with five arrays, where each of the five
    # elements is an array for number, BJD, uncertainty, epoch, DQ.
    # Yes, this is a dreadful situation to be in. A high priority todo is to rewrite these into "datapoint" objects
    # where each object stores a mid-transit time and uncertainty, its number, epoch, and DQ.
    return no, bjd, mid_err,epoch,dq

In [ ]:
def filter_etd_data(no, bjd, mid_err, epoch, dq, filter_dq):
    """Filters ETD data. Checks that uncertainty exists. Returns only datapoints with DQ equal to or better than specified value. N.B. 1 to 5 is best to worst
    Inputs:
    - no
    - bjd
    - mid_err
    - epoch
    - dq
    - filter_dq
    Outputs:
    - filteredData, the filtered data array (of arrays). Contains three arrays: BJD mid-time, the uncertainty, and the epoch"""

    # Filter for "good" data. I don't trust data without a tmid uncertainty, so check it exists/isn't 0
    good_data=[[],[],[],[],[]]

    for i in range(0, len(no)):
        if mid_err[i] > 0:
            good_data[0].append(no[i])
            good_data[1].append(bjd[i])
            good_data[2].append(mid_err[i])
            good_data[3].append(epoch[i])
            good_data[4].append(dq[i])

    entries = len(good_data[1]) # no of archive entries = length

    # make a blank array to put into. Note we arent using np.zeros because we dont know the final
    # length of the array as we dont know how many data points will pass the DQ filter.
    # TODO replace all np.zeroes usage with blank arrays
    # to recap: one array, contains three arrays - these three arrays are for mid, mid_err, epoch
    # we don't bother preserving no, it was just imported to be used basically as a debugging tool
    # and dq is pointless once we've filtered. This makes the addition of non-ETD data far easier
    filteredData = [[],[],[]]

    for i in range(0, entries): # for each entry in the data
        if good_data[4][i] <= filter_dq: # check the DQ vs the specified DQ argument. If better...

            # ...then add the data to each of the three arrays in filteredData that we care about
            filteredData[0].append(good_data[1][i])
            filteredData[1].append(good_data[2][i])
            filteredData[2].append(good_data[3][i])

    return filteredData # return the data array containing only the transits that's been filtered by DQ

## Statistics functions

These functions are for the $\chi^2$ comparison. I sincerely doubt that I haven't accidentally reinvented the wheel - functions for this probably already exist. However, it's so simple, I've not exactly wasted hours on these.

Equations are from "Measurements and Their Uncertainties: A Practical Guide to Modern Error Analysis: Hughes and Hase 2010".

$$ \chi^2 = \sum_i{\frac{y_i-y(x_i)}{\alpha_i^2}}$$

$$\chi^2_\text{reduced} = \frac{\chi^2}{\nu} $$

In [ ]:
def chi_squared(y,yx,alpha):
    """Calculates the unreduced chi squared.
    Inputs:
    - y, the actual observed y value (the y_i in the formula)
    - yx, the y value of the fitted line (the y(x) in the formula )
    - alpha, the error bar for y, in the same units as y
    Outputs:
    - chi2, the unreduced chi squared value
    """

    chi2=0
    for i in range(0,len(y)):
        chi2 += ((y[i]-yx[i])**2) / (alpha[i]**2)
    return chi2

def reduced_chi_squared(y,yx,alpha,m):
    """Calculates the reduced chi squared from the raw data: just chisq divided by degrees of freedom
    Degree of freedom is just number of observations n - number of fitted parameters m
    where n is just the number of y (or y(x) or x) values
    Inputs:
    - y, the actual observed y value (the y_i in the formula)
    - yx, the y value of the fitted line (the y(x) in the formula )
    - alpha, the error bar for y, in the same units as y
    - m, the number of fitted parameters.
    Outputs:
    - chi2, the unreduced chi squared value
    """
    n = len(y) # number of observations/datapoints
    dof = n-m # degree of freedom, m is fitted params
    chi2 = chi_squared(y,yx,alpha) # get the unreduced chi squared
    reduced_chi2 = chi2/dof # reduce it
    return reduced_chi2